# Capestone Project Solution2_3
## / Toxic Comment Classification /

- - -
<ul>
<li><a href="#prepare">I 环境准备</a></li>
<li><a href="#vecing">II 向量化</a></li>
<li><a href="#model">III 模型</a></li>
<li><a href="#conclusions">IV 结论</a></li>
</ul>

<a id='intro'></a>

<center><a id='prepare'>I 环境准备</a></center>

In [5]:
# 导入语句
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 导入机器学习库
from sklearn.model_selection import train_test_split

# 导入深度学习库
# from sklearn.datasets import load_files       
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.models import Sequential

from keras.layers import Embedding, Flatten, Dense, Dropout
#from keras.layers import Flatten
#from keras.layers import Dense
#from keras.layers import Dropout
from keras.layers import Conv1D, MaxPool1D, BatchNormalization

from keras.optimizers import Adam
from keras.regularizers import l2
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

from keras.models import model_from_json

# 设置参数显示长文本
pd.options.display.max_colwidth = 500

# 行内显示
%matplotlib inline

In [6]:
# 导入文件
test = pd.read_csv('test.csv')
train = pd.read_csv('train.csv')

# 检查标签
train.head(1)

id  \
0  0000997932d777bf   

                                                                                                                                                                                                                                                                comment_text  \
0  Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27   

   toxic  severe_toxic  obscene  threat  insult  identity_hate  
0      0             0        0       0       0              0

In [7]:
# get comment
train_comment = train.comment_text
test_comment = test.comment_text

In [8]:
# get lable
train_label = train.iloc[:,2:]
train_label.head(1)

toxic  severe_toxic  obscene  threat  insult  identity_hate
0      0             0        0       0       0              0

In [9]:
# check data
print(train_comment[0])
print(train_comment.shape)
print(test_comment[0])
print(test_comment.shape)
print(train_label.shape)

Explanation
Why the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27
(159571,)
Yo bitch Ja Rule is more succesful then you'll ever be whats up with you and hating you sad mofuckas...i should bitch slap ur pethedic white faces and get you to kiss my ass you guys sicken me. Ja rule is about pride in da music man. dont diss that shit on him. and nothin is wrong bein like tupac he was a brother too...fuckin white boys get things right next time.,
(153164,)
(159571, 6)


<center><a id='vecing'>II 向量化</a></center>

In [10]:
# --- 6B 300D ---
# 根据 solution2_test 文件，300D的成绩明显好于50D
# 也说明合适的维度非常重要

# 向量化词

max_feature = 10000
max_lenth = 100

## prepare tokenizer
t = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True, split=' ', num_words=max_feature )
## https://keras.io/preprocessing/text/
## 可以考虑直接去掉奇怪字符、lower
t.fit_on_texts(train_comment)
## 这里是对输入句子进行拆词
## 在这个数据里，如果输入了1000句（测试文件中的train_short)，结果是10007个
## 如果输出的话在1000之后会有个...
## 全部输入的话是21万多
## 可以使用 num_words 做限制（这里没有生效，后续研究）

vocab_size = len(t.word_index) + 1
## 根据上面设置 vocab_size

## integer encode the documents
encoded_train = t.texts_to_sequences(train_comment)
encoded_test = t.texts_to_sequences(test_comment)

## pad documents to a max length of 100 words
max_length = 300

## max_length 就是每个commet要处理的单词数
padded_train = pad_sequences(encoded_train, maxlen=max_length, padding='post')
padded_test = pad_sequences(encoded_test, maxlen=max_length, padding='post')

In [11]:
# check padded
print(padded_train.shape)
print(padded_train[99])

print(padded_test.shape)
print(padded_test[100])
## 可以看出来，100个词的 padded 数据，没有的用0填充了

(159571, 300)
[   6   40   33   42  173  282  145   89   26   22    6   96    5  672
  884   16  528    2   33   57    4   18   57 2292    8   39   73  371
    4    6  361    2   16  101 1463   21  567   37    6  292   18    5
  672   50   96   48   11 1018  439 3246    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0   

In [12]:
# check word list
len(t.word_index)
## 注意这里是先将词统计完的计数
## 与 max_length 无关，因为 pad_sequences 是后发生的

210337

<center><a id='model'>III 模型</a></center>

In [15]:
# 创建 embedding 层（300D）

embed_demention = 300

## 导入pre-training
embeddings_index = dict( )
f = open('glove.6B.300d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

## 创建嵌入矩阵
embedding_matrix = np.zeros((vocab_size, embed_demention))
## 注意这里要和嵌入矩阵的维度300相同
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

Loaded 400000 word vectors.


In [16]:
# check embedding_matrix
embedding_matrix.shape

(210338, 300)

In [21]:
# all class split 测试

# 使用 pretrained embedding 
# eporch 会影响准确度
# eporch5 和 eporch20 

# split train and val
x_train, x_val , y_train, y_val = train_test_split(padded_train, train_label, test_size=0.2, shuffle=True, random_state=42)

In [22]:
# from keras.layers import Convolution1D, MaxPool1D, Flatten, BatchNormalization

In [23]:
## model
model = Sequential()
## 设置embedding层
e = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=300, trainable=False)
model.add(e)

model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Conv1D(64, 5, padding='same', kernel_regularizer=l2(0.01)))
model.add(MaxPool1D())

model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Conv1D(128, 3, padding='same', kernel_regularizer=l2(0.01)))
model.add(MaxPool1D())

model.add(Flatten())
model.add(Dense(6, activation='sigmoid'))
# 6分类 Dense为6

## compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
## summarize the model
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 300, 300)          63101400  
_________________________________________________________________
dropout_1 (Dropout)          (None, 300, 300)          0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 300, 300)          1200      
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 300, 64)           96064     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 150, 64)           0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 150, 64)           0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 150, 64)           256       
__________

In [24]:
## fit the model
model.fit(x_train, y_train, batch_size=64, epochs=5, validation_data=(x_val, y_val), verbose=2)

Train on 127656 samples, validate on 31915 samples
Epoch 1/5
 - 2354s - loss: 0.1683 - acc: 0.9727 - val_loss: 0.1039 - val_acc: 0.9741
Epoch 2/5
 - 2346s - loss: 0.1027 - acc: 0.9748 - val_loss: 0.0925 - val_acc: 0.9752
Epoch 3/5
 - 2452s - loss: 0.0906 - acc: 0.9756 - val_loss: 0.0835 - val_acc: 0.9762
Epoch 4/5
 - 2485s - loss: 0.0852 - acc: 0.9760 - val_loss: 0.0855 - val_acc: 0.9750
Epoch 5/5
 - 2272s - loss: 0.0826 - acc: 0.9762 - val_loss: 0.0827 - val_acc: 0.9757


In [1]:
model.save_weights('s2_weight.h5')

NameError: name 'model' is not defined

In [ ]:
## Tip
### （在最开始的测试中）Tokenizer不做任何处理是 Accuracy: 95.334365
### 加了处理结果有一点上升 Accuracy: 95.417087
### 具有随机性

<center><a id='conclusions'>IV 结论</a></center>

In [ ]:
# output submission
y_target = model.predict(padded_test)
submission = pd.read_csv('sample_submission.csv')
submission[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]] = y_target
submission.to_csv('submission_s2.csv', index=False)

> 主要参考资料：
1. [项目建议中的LR + 词袋模式](https://www.kaggle.com/tunguz/logistic-regression-with-words-and-char-n-grams)
2. [Cross-validation Performance](https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation)

> 小结：
1. Solution1 为 LR + CBOW 的方式进行多分类计算
2. 输出结果是每个分类的可能性[0,1]

> Kaggle Score:
1. 0.97576